# Предсказание оттока

**Описание работы:**
&nbsp;&nbsp;&nbsp;&nbsp;
<br><br>
Любой бизнес хочет максимизировать количество клиентов. Для достижения этой цели важно не только пытаться привлечь новых, но и удерживать уже существующих. Удержать клиента обойдется компании дешевле, чем привлечь нового. Кроме того, новый клиент может оказаться слабо заинтересованным в услугах бизнеса и с ним будет сложно работать, тогда как о старых клиентах уже есть необходимые данные по взаимодействию с сервисом. 
 <br>

Соответственно, прогнозируя отток, мы можем вовремя среагировать и попытаться удержать клиента, который хочет уйти. Опираясь на данные об услугах, которыми пользуется клиент, мы можем сделать ему специальное предложение, пытаясь изменить его решение об уходе от оператора. Благодаря этому задача удержания будет легче в реализации, чем задача привлечения новых пользователей, о которых мы еще ничего не знаем.<br>

В данных содержится информация о почти шести тысячах пользователей, их демографических характеристиках, услугах, которыми они пользуются, длительности пользования услугами оператора, методе оплаты, размере оплаты. 
<br>

Cтоит задача проанализировать данные и спрогнозировать отток пользователей (выявить людей, которые продлят контракт и которые не продлят). 

_________

[Codebook](#Codebook) <br>
[1. Описание данных](#1.-Описание-данных)<br>
[2. Исследование зависимостей и формулирование гипотез](#2.-Исследование-зависимостей-и-формулирование-гипотез)<br>
[3. Построение моделей для прогнозирования оттока](#3.-Построение-моделей-для-прогнозирования-оттока)<br>
[4. Сравнение качества моделей](#4.-Сравнение-качества-моделей) <br>


In [1]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [3]:
from sklearn.metrics import accuracy_score, recall_score, classification_report, make_scorer, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm.sklearn import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
import numpy as np
from scipy.stats import boxcox
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [5]:
telecom_users = pd.read_csv('telecom_users.csv', index_col=0)
# Excluding users with zero-tenure
telecom_users = telecom_users.loc[telecom_users.tenure!=0] 
# (pd.to_numeric(telecom_users.TotalCharges)==1).sum()
# telecom_users.head()

In [6]:
# Переменные для графиков
GENERAL_COLUMNS = ['gender', 'SeniorCitizen', 'Partner', 'Dependents']
CONTRACT_COLUMNS = ['tenure', 'Contract', 'PaperlessBilling', 'PaymentMethod']
SERVICES_COLUMNS = [
    'PhoneService', 'MultipleLines', 'InternetService',
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
    'StreamingTV', 'StreamingMovies'
]
COLORS = ['rgba(152, 0, 0, .8)', 'rgba(255, 182, 193, .9)']
CHURN_STATUSES = telecom_users['Churn'].drop_duplicates()

# Преобразование числовых столбцов
make_float = lambda x: float(x) if x!=' ' else 0
telecom_users['TotalCharges'] = telecom_users['TotalCharges'].apply(make_float)
telecom_users['MonthlyCharges'] = telecom_users['MonthlyCharges'].apply(make_float)
#telecom_users['ARPU'] = telecom_users['TotalCharges'] / telecom_users['tenure']
#telecom_users = telecom_users.dropna()

## Codebook
<br>

[Ссылка для скачивания данных](https://drive.google.com/open?id=1dPCG76ST6NohYKtVMGv6HpFL-jD5p1eJ) 
<br><br>

`telecom_users.csv` содержит следующие значения:<br><br>

- `customerID` – id клиента<br>
- `gender` – пол клиента (male/female)<br>
- `SeniorCitizen` – яляется ли клиент пенсионером (1, 0)<br>
- `Partner` – состоит ли клиент в браке (Yes, No)<br>
- `Dependents` – есть ли у клиента иждивенцы (Yes, No)<br>
- `tenure` – сколько месяцев человек являлся клиентом компании<br>
- `PhoneService` – подключена ли услуга телефонной связи (Yes, No)<br>
- `MultipleLines` – подключены ли несколько телефонных линий (Yes, No, No phone service)<br>
- `InternetService` – интернет-провайдер клиента (DSL, Fiber optic, No)<br>
- `OnlineSecurity` – подключена ли услуга онлайн-безопасности (Yes, No, No internet service)<br>
- `OnlineBackup` – подключена ли услуга online backup (Yes, No, No internet service)<br>
- `DeviceProtection` – есть ли у клиента страховка оборудования (Yes, No, No internet service)<br>
- `TechSupport` – подключена ли услуга технической поддержки (Yes, No, No internet service)<br>
- `StreamingTV` – подключена ли услуга стримингового телевидения (Yes, No, No internet service)<br>
- `StreamingMovies` – подключена ли услуга стримингового кинотеатра (Yes, No, No internet service)<br>
- `Contract` – тип контракта клиента (Month-to-month, One year, Two year)<br>
- `PaperlessBilling` – пользуется ли клиент безбумажным биллингом (Yes, No)<br>
- `PaymentMethod` – метод оплаты (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))<br>
- `MonthlyCharges` – месячный размер оплаты на настоящий момент<br>
- `TotalCharges` – общая сумма, которую клиент заплатил за услуги за все время<br>
- `Churn` – произошел ли отток (Yes or No)<br>

# 1. Описание данных

In [7]:
fig = make_subplots(rows=2, 
                    cols=4, 
                    subplot_titles=tuple(GENERAL_COLUMNS+CONTRACT_COLUMNS)
                   )

for i, churn in enumerate(CHURN_STATUSES):
    data = telecom_users.loc[telecom_users['Churn'] == churn]
    for j, column in enumerate(GENERAL_COLUMNS):
        fig.add_trace(
            go.Histogram(
                x=data[column],
                marker_color=COLORS[i],
                name='',#churn,
                #hoverlabel='',
                hovertext=("Churned: " + churn),                
            ),
            row=1,
            col=j+1
        )

    for j, column in enumerate(CONTRACT_COLUMNS):
        fig.add_trace(
            go.Histogram(
                x=data[column],
                marker_color=COLORS[i],
                name='',#churn,
                hovertext="Churned: " + churn,
                #hoverlabel=''
            ),
            row=2,
            col=j+1
        )

fig.update_layout(template='plotly_dark', 
                  title='General distribution', 
                  showlegend=False,
                  barmode='stack',
                  height=800,
                  width=1200)

## Основные выводы из общего распределения подписчиков:

- <b>Выборка гендерно-сбалансирована</b>: мужчин незначительно больше, чем женщин. Оба пола уходят в отток примерно поровну
- <b>Пенсионеры составляют примерно 1/6 от общей выборки</b> и чуть больше 40% из них ушли в отток -  данную группу стоит выделить для более детального анализа
- <b>Семейных пользователей примерно одинаково с холостыми</b>, при этом важно отметить, что отток среди холостых пользователей значительно больше
- <b>Пользователей без иждевенцев примерно в 2,5 раза больше, чем без</b>, при этом у них отток существенно больше
- <b>Чем меньше срок жизни клиента - тем больше доля, ушедших в отток</b>
- <b>Месячные котракты - самый популярный тип контракта, однако около 40% пользователей этой группы ушли в отток</b>, при этом в двух других группах с длительными контрактами данный процент на порядок меньше
- <b>Безбумажный биллинг в полтора раза популярнее среди клиентов</b>, при это доля ушедших в отток почти в три раза больше для данной группы
- <b>Электронные чеки - самый популярный метод оплаты</b>, при этом в этой группе также самый высокий процент оттока

Таким образом, можно сказать, что <b>ключевыми признаками, влияющими на отток являются - срок жизни клиента, тип контракта, способ оплаты и биллинга</b>

In [8]:
fig = make_subplots(rows=3, 
                    cols=4, 
                    subplot_titles=tuple(SERVICES_COLUMNS)
                   )

for i, churn in enumerate(CHURN_STATUSES):
    data = telecom_users.loc[telecom_users['Churn'] == churn]
    for j, column in enumerate(SERVICES_COLUMNS):
        fig.add_trace(
            go.Histogram(
                x=data[column],
                marker_color=COLORS[i],
                name='',#churn,
                #hoverlabel='',
                hovertext=("Churned: " + churn),                
            ),
            row=j//4+1,
            col=j%4+1
        )
fig.update_layout(template='plotly_dark', 
                  showlegend=False, 
                  title='Split by services', 
                  barmode='stack',
                  height=1000,
                  width=1000)
fig.show()

## Основные выводы по использованию сервисов:

- Только у 10% пользователей были телефонные услуги
- Среди пользователей интернет-услуг наибольшее число ушедших в отток среди пользователей оптоволокна, также это самая группа пользователей
- Если у пользователя не подключен какой-либо доп сервис, то он с большей вероятностью ушел в отток

# 2. Исследование зависимостей и формулирование гипотез

## Базовый класс преобразований

In [9]:
BINARY_FEATURES = [
    # 'SeniorCitizen',
    # 'Partner', 'Dependents', 
    'PhoneService', 'MultipleLines', 'PaperlessBilling', 
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
    'StreamingTV', 'StreamingMovies'
]
BINARY_OPTIONS = {
    'Yes': 1,
    'No': 0,
    'No phone service': 0,
    'No internet service': 0
}
NUMERIC_FEATURES = [
    'tenure',
    'MonthlyCharges',
    'TotalCharges'
]
# TO_NORM = ['MonthlyCharges', ]
DUMMY_VARS = [
    # 'gender',
    'InternetService',
    'Contract',
    'PaymentMethod'
]
TARGET = 'Churn'

get_binary = lambda x: BINARY_OPTIONS[x] if type(x)==str else x

class TelecomUsersTransform():
    """
    Custom transform object for telecom users with following steps:
        1. Transform binary features into 1/0
        2. Transform numeric data into values
        3. Transform dummy variables
    """
    def __init__(self, binary_features, numeric_features, dummy_vars, features_to_norm, *, copy=True, clip=False):
        self.binary_features = binary_features
        self.numeric_features = numeric_features
        self.dummy_vars = dummy_vars
        self.columns_to_drop = None
        self.features_to_norm = features_to_norm
        self.copy = copy
        self.clip = clip

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None, **fit_params):
        X = X[self.binary_features + self.numeric_features + self.dummy_vars].copy()
        # Prepare binary values
        for binary_feature in self.binary_features:            
            X[binary_feature] = X[binary_feature].apply(get_binary)
        if y is not None:
            y = y.apply(get_binary)
        # Prepare numeric values
        for numeric_feature in self.numeric_features:
            fill_blanks = lambda x: 0 if x==' ' else x
            X[numeric_feature] = X[numeric_feature].apply(fill_blanks)
            X[numeric_feature] = pd.to_numeric(X[numeric_feature])
            # Linearization
            linearize = lambda x: 0 if x<=0 else np.log(x)
            if self.features_to_norm is not None and numeric_feature in self.features_to_norm:                
                # X[numeric_feature] = X[numeric_feature].apply(linearize)
                # https://towardsdatascience.com/box-cox-transformation-explained-51d745e34203
                X[numeric_feature] = boxcox(X[numeric_feature])[0]
        # Prepare dummy features
        for dummy in self.dummy_vars:
            X = X.merge(
                pd.get_dummies(X[dummy], prefix=dummy),
                how='left',
                left_index=True, right_index=True
            )
            X.drop(dummy, axis=1, inplace=True)        
        return X

    def fit_transform(self, X, y=None, **fit_params):
        self = self.fit(X, y)
        return self.transform(X, y, **fit_params)
    
    def inverse_transform(self, X, y=None):
        if X.name=='Churn':
            unbinary = lambda x: "Yes" if x==1 else "No"
            X = X.apply(unbinary)
        return X

In [10]:
# 
transformer = TelecomUsersTransform(binary_features=BINARY_FEATURES, numeric_features=NUMERIC_FEATURES, dummy_vars=DUMMY_VARS, features_to_norm=NUMERIC_FEATURES)
fig = px.imshow(
    pd.concat(
        [
            transformer.fit_transform(telecom_users), 
            telecom_users['Churn'].apply(get_binary)
        ], 
        axis=1
    ).corr(),
    title='Корреляция признаков друг с другом',
    template='plotly_dark',
    height=800,
    width=1200
)
fig.show()

In [11]:
fig = px.scatter(
    telecom_users,
    x='tenure',
    y='MonthlyCharges',
    color='Churn',
    template='plotly_dark',
    height=400,
    width=600,
    title='Месячные платежи vs Срок жизни клиента на сервисе'
)
fig.show()

## Выводы по зависимостям

- Самая высокая зависимость признака оттока с типом контракта и длительностью подписки
- Также зависимость наблюдается с наличием у пользователя доп. сервисов - Online Security, Tech Support, Online Backup, Device Protection
- Также зависимость наблюдается с месячными платежами и общей суммой платежей
- Самая низкая зависимость с <b>полом, наличием телефонных, интернет услуг и мультиканальной телефонной связи - данные признаки будут исключены из модели</b>
- Отдельно хотелось бы отметить нелинейную связь оттока с длительностью жизни подписчика на сервисе и месячных платежей

## Основные гипотезы:

- В качестве ключевой метрики при выборе модели, на мой взгляд, нужно использовать полноту (recall), т.е. необходимо минимизировать количество ложно-положительных предсказаний (т.е. когда класс - отток, модель должна давать минимальную ошибку и не предсказывать обратное)
- Поскольку выборка состоит преимущественно из бинарных признаков (да/нет), то наиболее простым решением в данном случае будет построить модель классификации на базе Байесовского метода и деревьев решений
- Из-за большого количества несвязанных признаков свою эффективность может показать также алгоритм лог-регрессии и бустинга
- Также, на мой взгляд, имеет смысл попробовать нелинейные методы деревьев решений и бустинга с помощью моделей xgboost и lightgbm
- Отдельно, ради эксперимента можно попробовать объединение моделей в ансамбль, где на первом этапе попробовать отделить выбросы при помощи K-ближайших и далее на отфильтрованных данных обучать модель, которая получит лучший показатель полноты (recall) на предыдущих этапах
- Дополнительно можно применить стандартный классификатор на базе нейронных сетей из реализации sklearn

# 3. Построение моделей для прогнозирования оттока

In [12]:
def show_report(y_true, y_predict, y_proba, is_test=False):
    print(f'{"Train" if not is_test else "Test"}:\n', classification_report(y_true, y_predict))
    print('ROC_AUC test:', np.round(roc_auc_score(y_true, y_proba), 4))
    print()

In [13]:
X_train, X_test = train_test_split(telecom_users.set_index('customerID'), shuffle=True, test_size=0.2, random_state=33)
y_train = X_train[TARGET]
y_test = X_test[TARGET]
X_train.drop(TARGET, axis=1, inplace=True)
X_test.drop(TARGET, axis=1, inplace=True)

## Наивный Байес

In [14]:
gaussian_model = GaussianNB()
gaussian_pipeline = Pipeline(
    [
        ('TelecomUsersTranform', TelecomUsersTransform(binary_features=BINARY_FEATURES, numeric_features=NUMERIC_FEATURES, dummy_vars=DUMMY_VARS, features_to_norm=NUMERIC_FEATURES)),
        ('gaussian', gaussian_model)
    ]
)
gaussian_pipeline = gaussian_pipeline.fit(X_train, y_train)

y_pred = gaussian_pipeline.predict(X_train)
show_report(y_train, y_pred, gaussian_pipeline.predict_proba(X_train)[:,1])
# Оценка на тестовой выборке
y_pred = gaussian_pipeline.predict(X_test)
show_report(y_test, y_pred, gaussian_pipeline.predict_proba(X_test)[:,1], True)

Train:
               precision    recall  f1-score   support

          No       0.90      0.73      0.80      3516
         Yes       0.51      0.78      0.61      1264

    accuracy                           0.74      4780
   macro avg       0.70      0.75      0.71      4780
weighted avg       0.80      0.74      0.75      4780

ROC_AUC test: 0.8327

Test:
               precision    recall  f1-score   support

          No       0.90      0.75      0.82       873
         Yes       0.53      0.77      0.63       323

    accuracy                           0.75      1196
   macro avg       0.71      0.76      0.72      1196
weighted avg       0.80      0.75      0.76      1196

ROC_AUC test: 0.8421



## DecisionTree

In [15]:
decision_tree_model = DecisionTreeClassifier(random_state=33)
decision_tree_pipeline = Pipeline(
    [
        ('TelecomUsersTranform', TelecomUsersTransform(binary_features=BINARY_FEATURES, numeric_features=NUMERIC_FEATURES, dummy_vars=DUMMY_VARS, features_to_norm=NUMERIC_FEATURES)),
        ('decision_tree', decision_tree_model)
    ]
)
decision_tree_pipeline = decision_tree_pipeline.fit(X_train, y_train)

y_pred = decision_tree_pipeline.predict(X_train)
show_report(y_train, y_pred, decision_tree_pipeline.predict_proba(X_train)[:,1])
# Оценка на тестовой выборке
y_pred = decision_tree_pipeline.predict(X_test)
show_report(y_test, y_pred, decision_tree_pipeline.predict_proba(X_test)[:,1], True)

Train:
               precision    recall  f1-score   support

          No       1.00      1.00      1.00      3516
         Yes       1.00      0.99      0.99      1264

    accuracy                           1.00      4780
   macro avg       1.00      0.99      1.00      4780
weighted avg       1.00      1.00      1.00      4780

ROC_AUC test: 1.0

Test:
               precision    recall  f1-score   support

          No       0.80      0.82      0.81       873
         Yes       0.48      0.44      0.46       323

    accuracy                           0.72      1196
   macro avg       0.64      0.63      0.64      1196
weighted avg       0.71      0.72      0.72      1196

ROC_AUC test: 0.6319



## Логистическая регрессия

In [16]:
log_reg_pipeline = Pipeline(
    [
        ('TelecomUsersTranform', TelecomUsersTransform(binary_features=BINARY_FEATURES, 
                                                       numeric_features=NUMERIC_FEATURES, 
                                                       dummy_vars=DUMMY_VARS, 
                                                       features_to_norm=NUMERIC_FEATURES)),
        ('log_reg', LogisticRegression(
            C=2,
            fit_intercept=False,
            max_iter=500, 
            penalty='l2',
            tol=1e-3,
            # https://towardsdatascience.com/why-weight-the-importance-of-training-on-balanced-datasets-f1e54688e7df
            class_weight='balanced'))
    ]
)
log_reg_pipeline = log_reg_pipeline.fit(X_train, y_train)

y_pred = log_reg_pipeline.predict(X_train)
show_report(y_train, y_pred, log_reg_pipeline.predict_proba(X_train)[:,1])
# Оценка на тестовой выборке
y_pred = log_reg_pipeline.predict(X_test)
show_report(y_test, y_pred, log_reg_pipeline.predict_proba(X_test)[:,1], True)

Train:
               precision    recall  f1-score   support

          No       0.91      0.74      0.81      3516
         Yes       0.52      0.79      0.63      1264

    accuracy                           0.75      4780
   macro avg       0.72      0.77      0.72      4780
weighted avg       0.81      0.75      0.77      4780

ROC_AUC test: 0.8468

Test:
               precision    recall  f1-score   support

          No       0.87      0.84      0.85       873
         Yes       0.60      0.67      0.63       323

    accuracy                           0.79      1196
   macro avg       0.74      0.75      0.74      1196
weighted avg       0.80      0.79      0.79      1196

ROC_AUC test: 0.8503



In [22]:
skf = StratifiedKFold(n_splits=5)

parameters = {
    'log_reg__penalty': ('l1', 'l2', 'elasticnet', 'none'),
    'log_reg__tol': (1e-4, 1e-3, 1e-2),
    'log_reg__C': (1, 0.5, 0.05, 2, 4),
    'log_reg__fit_intercept': (True, False),
    'log_reg__max_iter': (500, 600),
}
clf = GridSearchCV(
    log_reg_pipeline, 
    parameters,
    scoring='roc_auc',
    cv=skf,
    n_jobs=-1
    )
clf.fit(X_train, y_train)
print(f'Лучшая модель дает score: {clf.best_score_}\n')
clf.best_params_

Лучшая модель дает score: 0.8426890159522097

/Users/dbevz/Documents/skillbox/skillbox/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning:

One or more of the test scores are non-finite: [       nan        nan        nan 0.84198648 0.84198648 0.84198648
        nan        nan        nan 0.78155885 0.78155885 0.78155885
        nan        nan        nan 0.84198648 0.84198648 0.84198648
        nan        nan        nan 0.78155885 0.78155885 0.78155885
        nan        nan        nan 0.84010719 0.84010719 0.84010719
        nan        nan        nan 0.77880947 0.77880947 0.77880947
        nan        nan        nan 0.84010719 0.84010719 0.84010719
        nan        nan        nan 0.77880947 0.77880947 0.77880947
        nan        nan        nan 0.83952801 0.83952801 0.83952801
        nan        nan        nan 0.78155885 0.78155885 0.78155885
        nan        nan        nan 0.83954375 0.83954375 0.83954375
        nan        nan        nan 0.78155885 0.

{'log_reg__C': 2,
 'log_reg__fit_intercept': False,
 'log_reg__max_iter': 500,
 'log_reg__penalty': 'l2',
 'log_reg__tol': 0.0001}

## XGBooster

In [17]:
xgb_pipeline = Pipeline(
    [
        ('TelecomUsersTranform', TelecomUsersTransform(binary_features=BINARY_FEATURES, numeric_features=NUMERIC_FEATURES, dummy_vars=DUMMY_VARS, features_to_norm=NUMERIC_FEATURES)),
        ('xgb', XGBClassifier(n_estimators=100, 
                    learning_rate=0.1,
                    objective='binary:logistic',
                    scale_pos_weight=0.01,
                    eval_metric='merror',
                    use_label_encoder=True))
    ]
)
xgb_pipeline = xgb_pipeline.fit(X_train, y_train)

y_pred = xgb_pipeline.predict(X_train)
show_report(y_train, y_pred, xgb_pipeline.predict_proba(X_train)[:,1])
# Оценка на тестовой выборке
y_pred = xgb_pipeline.predict(X_test)
show_report(y_test, y_pred, xgb_pipeline.predict_proba(X_test)[:,1], True)

/Users/dbevz/Documents/skillbox/skillbox/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

/Users/dbevz/Documents/skillbox/skillbox/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/dbevz/Documents/skillbox/skillbox/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Us

In [25]:
skf = StratifiedKFold(n_splits=5)

parameters = {
    'xgb__n_estimators': (50, 100, 200, 400, 800),
    'xgb__learning_rate': (0.01, 0.1, 1, 0.001),
    'xgb__max_depth': (5, 7, 10, 15, 20),
    'xgb__scale_pos_weight': (0.01, 0.1, 1, 0.001),
}
clf = GridSearchCV(
    xgb_pipeline, 
    parameters,
    scoring='roc_auc',
    cv=skf,
    n_jobs=-1
    )
clf.fit(X_train, y_train)
print(f'Лучшая модель дает score: {clf.best_score_}\n')
clf.best_params_

Лучшая модель дает score: 0.8404843903780188

/Users/dbevz/Documents/skillbox/skillbox/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



{'xgb__learning_rate': 0.1,
 'xgb__max_depth': 7,
 'xgb__n_estimators': 100,
 'xgb__scale_pos_weight': 0.01}

## lightgbm

In [18]:
lightgbm_pipeline = Pipeline(
    [
        ('TelecomUsersTranform', TelecomUsersTransform(binary_features=BINARY_FEATURES, numeric_features=NUMERIC_FEATURES, dummy_vars=DUMMY_VARS, features_to_norm=NUMERIC_FEATURES)),
        ('lightgbm', LGBMClassifier(
            boosting_type='goss',
            learning_rate=0.01,
            max_depth=5,
            n_estimators=300,
            num_leaves=12
        ))
    ]
)
lightgbm_pipeline = lightgbm_pipeline.fit(X_train, y_train)

y_pred = lightgbm_pipeline.predict(X_train)
show_report(y_train, y_pred, lightgbm_pipeline.predict_proba(X_train)[:,1])
# Оценка на тестовой выборке
y_pred = xgb_pipeline.predict(X_test)
show_report(y_test, y_pred, lightgbm_pipeline.predict_proba(X_test)[:,1], True)

Train:
               precision    recall  f1-score   support

          No       0.85      0.92      0.88      3516
         Yes       0.71      0.54      0.61      1264

    accuracy                           0.82      4780
   macro avg       0.78      0.73      0.75      4780
weighted avg       0.81      0.82      0.81      4780

ROC_AUC test: 0.8674

Test:
               precision    recall  f1-score   support

          No       0.73      1.00      0.84       873
         Yes       0.00      0.00      0.00       323

    accuracy                           0.73      1196
   macro avg       0.36      0.50      0.42      1196
weighted avg       0.53      0.73      0.62      1196

ROC_AUC test: 0.844

/Users/dbevz/Documents/skillbox/skillbox/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [32]:
skf = StratifiedKFold(n_splits=5)

parameters = {
    'lightgbm__boosting_type': ('gbdt', 'dart', 'goss', 'rf'),
    'lightgbm__num_leaves': (31, 62, 93, 12),
    'lightgbm__max_depth': (5, 7, 10, 15, 20, -1),
    'lightgbm__learning_rate': (0.01, 0.1, 1, 0.001),
    'lightgbm__n_estimators': (100, 200, 300, 400, 500),
}
clf = GridSearchCV(
    lightgbm_pipeline, 
    parameters,
    scoring='roc_auc',
    cv=skf,
    n_jobs=-1
    )
clf.fit(X_train, y_train)
print(f'Лучшая модель дает score: {clf.best_score_}\n')
clf.best_params_

Лучшая модель дает score: 0.842636223595713



{'lightgbm__boosting_type': 'goss',
 'lightgbm__learning_rate': 0.01,
 'lightgbm__max_depth': 5,
 'lightgbm__n_estimators': 300,
 'lightgbm__num_leaves': 12}

# 4. Сравнение качества моделей 

In [19]:
import warnings
warnings.simplefilter("ignore", UserWarning)

In [36]:
telecom_users_pipeline = Pipeline(
    [
        ('TelecomUsersTranform', TelecomUsersTransform(binary_features=BINARY_FEATURES, numeric_features=NUMERIC_FEATURES, dummy_vars=DUMMY_VARS, features_to_norm=NUMERIC_FEATURES)),
        ('model', GaussianNB())
    ]
)

gaussian_model = gaussian_model
log_reg_model = log_reg_pipeline['log_reg']
xgb_model = xgb_pipeline['xgb']
lgbm_model = lightgbm_pipeline['lightgbm']

skf = StratifiedKFold(n_splits=5)

parameters = {
    'model': (gaussian_model, log_reg_model, xgb_model, lgbm_model),
}
clf = GridSearchCV(
    telecom_users_pipeline, 
    parameters,
    scoring='roc_auc',
    cv=skf
    )
clf.fit(X_train, y_train)
clf.best_estimator_

Pipeline(steps=[('TelecomUsersTranform',
                 <__main__.TelecomUsersTransform object at 0x7fb5c3c32ef0>),
                ('model',
                 LogisticRegression(C=2, class_weight='balanced',
                                    fit_intercept=False, max_iter=500,
                                    tol=0.001))])

In [42]:
compare = pd.DataFrame(clf.cv_results_)
compare['param_model'] = compare['param_model'].apply(str)

In [51]:
px.bar(
    compare.sort_values('mean_test_score'),
    x='param_model',
    y='mean_test_score',
    template='plotly_dark',
    height=1000
)

In [20]:
import pickle


In [32]:
with open('/Users/dbevz/Documents/skillbox/Datascientist/Machine Learning/Coursework/dbevz_model.pickle', 'wb') as f:
    pickle.dump(log_reg_pipeline, f)

In [34]:
with open('/Users/dbevz/Documents/skillbox/Datascientist/Machine Learning/Coursework/dbevz_model.pickle', 'rb') as f:
    loaded_obj = pickle.load(f)

In [36]:
loaded_obj.predict(X_test)

array(['No', 'No', 'No', ..., 'Yes', 'No', 'No'], dtype=object)